In [16]:
import Turtle_trading
import pandas as pd 
import numpy as np


In [52]:
class Account_Handler:
    NO_OF_STOCK_TYPE = 2
    account_size = 100000

    def __init__(self,risk_percentage):
        # self.account_size = account_size
        self.risk_percentage = risk_percentage/100
        self.trading_amt_per_share = self.account_size/self.NO_OF_STOCK_TYPE
        self.risk = self.trading_amt_per_share * self.risk_percentage

    def ATR(self, filename, ticker):
        ticker = ticker.upper()
        xls = pd.ExcelFile(filename)
        if ticker in xls.sheet_names:
            df = pd.read_excel(filename, sheet_name=ticker)

            # Ensure Date column is datetime and set as index
            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)

            last_row = df.iloc[-1]
            try:
                ATR = last_row['Avg_True_Range(N)']
            except KeyError as msg:
                return msg
            
            return ATR
        else:
            return f"Ticker {ticker} not found in the Excel file."

    def cal_positon_size(self,ATR):
        """Calculate unit for trading(No. of stock)"""
        # ATR = Account_Handler.ATR(self, filename= filename,ticker= ticker)
        position_size = self.risk // ATR
        return int(position_size)   

    def cal_entry_prize(self, filename, ticker):
        """We have to bring this from Actual Buying Prize. But for now we will consider it as previous days close."""
        ticker = ticker.upper()
        xls = pd.ExcelFile(filename)
        if ticker in xls.sheet_names:
            df = pd.read_excel(filename, sheet_name=ticker)

            # Ensure Date column is datetime and set as index
            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)

            last_row = df.iloc[-1]
            try:
                entry_prize = last_row['Close']
            except KeyError as msg:
                return msg
            return entry_prize
        else:
            return f"Ticker {ticker} not found in the Excel file."
        
    def stop_loss_entry(self, entry_prize, ATR):
        self.stop_loss_prize = entry_prize - 2*ATR
        return self.stop_loss_prize
    
    def first_pyr_sig_prize(self, entry_prize, ATR):
        self.first_pyr_prize = entry_prize + 0.5*ATR
        return self.first_pyr_prize
    
    def stop_loss_pyr_1(self, stop_loss_entry, ATR):
        self.stop_loss_prize = stop_loss_entry + 0.5*ATR
        return self.stop_loss_prize
    
    def second_pyr_sig_prize(self, entry_prize, ATR):
        self.first_pyr_prize = entry_prize + ATR
        return self.first_pyr_prize
    
    def stop_loss_pyr_2(self, stop_loss_entry, ATR):
        self.stop_loss_prize = stop_loss_entry + ATR
        return self.stop_loss_prize


In [53]:
filename = r"C:\Users\DELL\open_source\Turtle_trading\123.xlsx"

A1 = Account_Handler(risk_percentage=1)
ATR = A1.ATR(filename= filename, ticker="MOTILALOFS")
unit = A1.cal_positon_size(ATR= ATR)
entry_prize = A1.cal_entry_prize(filename= filename, ticker="MOTILALOFS")
stop_loss = A1.stop_loss_entry(entry_prize, ATR)
first_pyr_entry_prize = A1.first_pyr_sig_prize(entry_prize, ATR)
stop_loss_pyr_1 = A1.stop_loss_pyr_1(stop_loss,ATR)
second_pyr_sig_prize = A1.second_pyr_sig_prize(entry_prize, ATR)
stop_loss_pyr_2 = A1.stop_loss_pyr_2(stop_loss, ATR)

print(ATR, unit, entry_prize, stop_loss,)
print( first_pyr_entry_prize, stop_loss_pyr_1, second_pyr_sig_prize, stop_loss_pyr_2)

49.11 10 1029.7 931.48
1054.255 956.035 1078.81 980.59
